In [1]:
!pip install pytorch-lightning
PATH = '/kaggle/input/nowcasting-yandex-cup/'

In [3]:
import h5py
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import tqdm
import pytorch_lightning as L

In [4]:
from pytorch_lightning import seed_everything
import random
seed=7
seed_everything(seed, workers=True)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [47]:
class RadarDataset(data.Dataset):

    def __init__(self, list_of_files, in_seq_len=4, out_seq_len=12, mode='overlap', rotate = 0, with_time=False):
        self.in_seq_len = in_seq_len
        self.out_seq_len = out_seq_len
        self.seq_len = in_seq_len + out_seq_len
        self.with_time = with_time
        self.__prepare_timestamps_mapping(list_of_files)
        self.__prepare_sequences(mode)
        self.rotate = rotate

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, index):
        to_append = []
        data = []
        targets = []
        for timestamp in self.sequences[index]:
            with h5py.File(self.timestamp_to_file[timestamp]) as d:
                targets.append(np.array(torch.rot90(torch.tensor(np.array([d[timestamp]['intensity']])), self.rotate)))
                data.append(np.array([d[timestamp]['intensity'], d[timestamp]['reflectivity'][0], d[timestamp]['reflectivity'][1], d[timestamp]['reflectivity'][2], 
                                      d[timestamp]['reflectivity'][3], d[timestamp]['reflectivity'][4], d[timestamp]['reflectivity'][5],
                                     d[timestamp]['reflectivity'][6], d[timestamp]['reflectivity'][7],
                                     d[timestamp]['radial_velocity'][0], d[timestamp]['radial_velocity'][1], d[timestamp]['radial_velocity'][2],
                                     d[timestamp]['radial_velocity'][3], d[timestamp]['radial_velocity'][4],
                                     d[timestamp]['radial_velocity'][5], d[timestamp]['radial_velocity'][6], d[timestamp]['radial_velocity'][7]]))
                #data.append(np.array([d[timestamp]['intensity'], d[timestamp]['events'], d[timestamp]['reflectivity'][0]]))
                
        data = np.array(data)
        targets = np.array(targets)
        #data = np.expand_dims(data, axis=1)
        #targets = np.expand_dims(targets, axis=1)
        data[data == -1e6] = 0
        data[data == -2e6] = -1
        targets[targets == -1e6] = 0
        targets[targets == -2e6] = -1
        inputs = data[:self.in_seq_len]
        targets = targets[self.in_seq_len:]
        if self.with_time:
            return (inputs, self.sequences[index][-1]), targets
        else:
            return inputs, targets

    def __prepare_timestamps_mapping(self, list_of_files):
        self.timestamp_to_file = {}
        for filename in list_of_files:
            with h5py.File(filename) as d:
                self.timestamp_to_file = {
                    **self.timestamp_to_file,
                    **dict(map(lambda x: (x, filename), d.keys()))
                }

    def __prepare_sequences(self, mode):
        timestamps = np.unique(sorted(self.timestamp_to_file.keys()))
        if mode == 'sequentially':
            self.sequences = [
                timestamps[index * self.seq_len: (index + 1) * self.seq_len]
                for index in range(len(timestamps) // self.seq_len)
            ]
        elif mode == 'overlap':
            self.sequences = [
                timestamps[index: index + self.seq_len]
                for index in range(len(timestamps) - self.seq_len + 1)
            ] 
        else:
            raise Exception(f'Unknown mode {mode}')
        self.sequences = list(filter(
            lambda x: int(x[-1]) - int(x[0]) == (self.seq_len - 1) * 600,
            self.sequences
        ))

In [46]:
class ConvLSTMCell(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, padding, activation):
        super().__init__()

        if activation == 'tanh':
            self.activation = torch.tanh
        elif activation == 'relu':
            self.activation = torch.relu

        self.conv_int_0 = nn.Conv2d(
            in_channels=in_channels + out_channels,
            out_channels= 4 * out_channels,
            kernel_size=kernel_size,
            padding=padding
        )
        
        self.conv_int_1 = nn.Conv2d(
            in_channels= 4 * out_channels,
            out_channels=4 * out_channels,
            kernel_size=kernel_size,
            padding=padding
        )
        self.conv_refl_0 = nn.Conv2d(
            in_channels=16 + out_channels,
            out_channels= 4 * out_channels,
            kernel_size=kernel_size,
            padding=padding
        )
        
    def forward(self, X, H_prev, C_prev):
        
        int_X = X[:,0:1, :, :]
        refl_X= X[:,1:, :, :]
        H_prev_int, H_prev_refl = torch.chunk(H_prev, chunks=2, dim=1)
        conv_int_output = self.conv_int_1(self.activation(self.conv_int_0(torch.cat([int_X, H_prev_int], dim=1))))
        conv_refl_output = self.conv_refl_0(torch.cat([refl_X, H_prev_refl], dim=1))
        
        i_conv_int, f_conv_int, C_conv_int, o_conv_int = torch.chunk(conv_int_output, chunks=4, dim=1)
        i_conv_refl, f_conv_refl, C_conv_refl, o_conv_refl = torch.chunk(conv_refl_output, chunks=4, dim=1)


        input_gate = torch.sigmoid(torch.cat([i_conv_int, i_conv_refl], dim=1))
        forget_gate = torch.sigmoid(torch.cat([f_conv_int, f_conv_refl], dim=1))
        output_gate = torch.sigmoid(torch.cat([o_conv_int, o_conv_refl], dim=1))
        C_conv = torch.cat([C_conv_int, C_conv_refl], dim=1)
        C = forget_gate * C_prev + input_gate * self.activation(C_conv)
        H = output_gate * self.activation(C)
        return H, C


class ConvLSTM(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, padding, activation):
        super().__init__()
        self.out_channels = out_channels
        self.convLSTMCell = ConvLSTMCell(in_channels, out_channels, kernel_size, padding, activation)

    def forward(self, X):
        batch_size, seq_len, _, height, width = X.size()
        output = torch.zeros(batch_size, seq_len, 2 * self.out_channels, height, width, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        H = torch.zeros(batch_size, 2 * self.out_channels, height, width, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        C = torch.zeros(batch_size, 2 * self.out_channels, height, width, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        for time_step in range(seq_len):
            H, C = self.convLSTMCell(X[:, time_step], H, C)
            output[:, time_step] = H
        return output


class Seq2Seq(nn.Module):

    def __init__(
        self, num_channels, num_kernels, kernel_size, padding, activation, num_layers, out_seq_len
    ):
        super().__init__()
        self.out_seq_len = out_seq_len

        self.activation = torch.relu
        self.sequential = nn.Sequential()
        self.sequential.add_module(
            'convlstm1',
            ConvLSTM(
                in_channels=num_channels,
                out_channels=num_kernels,
                kernel_size=kernel_size,
                padding=padding,
                activation=activation
            )
        )
        for layer_index in range(2, num_layers + 1):
            self.sequential.add_module(
                f'convlstm{layer_index}',
                ConvLSTM(
                    in_channels=num_kernels,
                    out_channels=num_kernels,
                    kernel_size=kernel_size,
                    padding=padding,
                    activation=activation
                )
            )
        self.conv = nn.Conv2d(
            in_channels=2 * num_kernels,
            out_channels=num_channels,
            kernel_size=kernel_size,
            padding=padding
        )

    def forward(self, X):
        batch_size, seq_len, num_channels, height, width = X.size()
        inputs = torch.zeros(
            batch_size, seq_len + self.out_seq_len - 1, num_channels, height, width,
            device=self.conv.weight.device
        )
        inputs[:, :seq_len] = X
        output = self.sequential(inputs)
        output = torch.stack([
            self.conv(output[:, index + seq_len - 1])
            for index in range(self.out_seq_len)
        ], dim=1)
        return output


class ConvLSTMModel(L.LightningModule):

    def __init__(self):
        super().__init__()
        self.model = Seq2Seq(
            num_channels=1,
            num_kernels=32,
            kernel_size=(3, 3),
            padding=(1, 1),
            activation='relu',
            num_layers=1,
            out_seq_len=12
        )

    def forward(self, x):
        x = x.to(device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        output = self.model(x)
        return output

    def training_step(self, batch):
        x, y = batch
        out = self.forward(x)
        out[y == -1] = -1
        loss = F.mse_loss(out, y)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=3e-4)
        return optimizer

In [48]:
def prepare_summer_loaders(train_batch_size=1): #summer up
    train_datasets = []
    val_datasets = []
    for i in range(5, 9):
        month = '0' + str(i)
        path = PATH + '2021-' + month + '-train.hdf5'
        month_dataset = RadarDataset([path])
        #train_month, val_month = torch.utils.data.random_split(month_dataset, [0.8, 0.2])
        train_datasets.append(month_dataset)
        #val_datasets.append(val_month)
    full_train_dataset = torch.utils.data.ConcatDataset([train_datasets[i] for i in range(len(train_datasets))])
    train_loader = data.DataLoader(full_train_dataset, batch_size=train_batch_size, shuffle=True, num_workers=4)
    #val_loaders = [data.DataLoader(val_datasets[i], batch_size=train_batch_size, shuffle=False, num_workers=4) for i in range(len(val_datasets))] 
    return train_loader#, val_loaders

In [49]:
summer_train = prepare_summer_loaders()  #, summer_vals
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [51]:
summer_model = ConvLSTMModel()

In [52]:
summer_model.current_lr = 3e-4
trainer = L.Trainer(
    max_epochs=1
)
summer_model.to(device)
trainer.fit(summer_model, summer_train)

Training: |          | 0/? [00:00<?, ?it/s]

In [ ]:
def evaluate_on_val(model, valid_loader):
    rmses = np.zeros((12,), dtype=float)
    for item in tqdm.tqdm(valid_loader):
        inputs, target = item
        output = model(inputs)
        
        rmses += np.sum((
            np.square(target.detach().cpu().numpy() - output.detach().cpu().numpy())
        ) * (target.detach().cpu().numpy() != -1), axis=(0, 2, 3, 4))
    rmses /= len(valid_loader)
    return np.mean(np.sqrt(rmses))

In [140]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
summer_model.to(device) #5e-4 + 3e-4
for month in range(5, 9):
    print('month', month)
    print(evaluate_on_val(summer_model, summer_vals[month - 5]))

In [45]:
torch.save(summer_model, 'summer_no_val_8e-4')